In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def get_economic_cycle_indicators():
    """
    Coleta indicadores econômicos para determinar o ciclo vigente
    """
    # Período de análise (últimos 2 anos)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)
    
    indicators = {}
    
    try:
        print("Coletando dados dos Treasuries...")
        # 1. Curva de Juros (10Y-2Y Treasury Spread)
        treasury_10y = yf.download("^TNX", start=start_date, end=end_date, progress=False)
        treasury_2y = yf.download("^IRX", start=start_date, end=end_date, progress=False)
        
        if not treasury_10y.empty and not treasury_2y.empty and 'Close' in treasury_10y.columns and 'Close' in treasury_2y.columns:
            # Pegar apenas os valores válidos (não-NaN)
            t10_clean = treasury_10y['Close'].dropna()
            t2_clean = treasury_2y['Close'].dropna()
            
            if len(t10_clean) > 0 and len(t2_clean) > 0:
                # Usar o último valor disponível
                yield_10y = float(t10_clean.iloc[-1])
                yield_2y = float(t2_clean.iloc[-1])
                
                indicators['yield_curve_current'] = yield_10y - yield_2y
                indicators['yield_curve_trend'] = 'Invertida' if indicators['yield_curve_current'] < 0 else 'Normal'
                print(f"✅ Treasuries coletados: 10Y={yield_10y:.2f}%, 2Y={yield_2y:.2f}%")
            else:
                print("⚠️ Dados de Treasury sem valores válidos")
        else:
            print("⚠️ Erro ao baixar dados de Treasury")
        
        # 2. S&P 500 Performance e Volatilidade
        sp500 = yf.download("^GSPC", start=start_date, end=end_date, progress=False)
        if not sp500.empty:
            sp500_returns = sp500['Close'].pct_change().dropna()
            
            indicators['sp500_ytd_return'] = float(((sp500['Close'].iloc[-1] / sp500['Close'].iloc[0]) - 1) * 100)
            indicators['sp500_volatility'] = float(sp500_returns.std() * np.sqrt(252) * 100)  # Anualizada
        
        # 3. VIX (Fear Index)
        vix = yf.download("^VIX", start=start_date, end=end_date, progress=False)['Close']
        if not vix.empty:
            indicators['vix_current'] = float(vix.iloc[-1])
            indicators['vix_avg_3m'] = float(vix.tail(63).mean()) if len(vix) >= 63 else np.nan
        
        # 4. Dollar Index (DXY)
        dxy = yf.download("DX-Y.NYB", start=start_date, end=end_date, progress=False)['Close']
        if not dxy.empty:
            indicators['dxy_current'] = float(dxy.iloc[-1])
            indicators['dxy_change_3m'] = float(((dxy.iloc[-1] / dxy.iloc[-63]) - 1) * 100) if len(dxy) >= 63 else np.nan
        
        # 5. Commodities (Oil - WTI)
        oil = yf.download("CL=F", start=start_date, end=end_date, progress=False)['Close']
        if not oil.empty:
            indicators['oil_current'] = float(oil.iloc[-1])
            indicators['oil_change_3m'] = float(((oil.iloc[-1] / oil.iloc[-63]) - 1) * 100) if len(oil) >= 63 else np.nan
        
        # 6. High Yield vs Treasury Spread
        hy_bond = yf.download("HYG", start=start_date, end=end_date, progress=False)['Close']
        treasury_etf = yf.download("IEF", start=start_date, end=end_date, progress=False)['Close']
        
        if not hy_bond.empty and not treasury_etf.empty:
            hy_returns = hy_bond.pct_change().dropna()
            treasury_returns = treasury_etf.pct_change().dropna()
            
            # Correlação como proxy para spread de crédito
            if len(hy_returns) > 30 and len(treasury_returns) > 30:
                common_dates = hy_returns.index.intersection(treasury_returns.index)
                if len(common_dates) > 30:
                    correlation = hy_returns[common_dates].corr(treasury_returns[common_dates])
                    indicators['credit_risk_signal'] = 'Alto' if correlation < -0.3 else 'Baixo'
        
    except Exception as e:
        print(f"Erro ao coletar dados: {e}")
    
    return indicators

def determine_economic_cycle(indicators):
    """
    Determina o ciclo econômico baseado nos indicadores
    """
    score = 0
    signals = []
    
    # Análise da Curva de Juros
    if 'yield_curve_current' in indicators:
        if indicators['yield_curve_current'] < -0.5:
            score -= 2
            signals.append("⚠️ Curva de juros invertida (recessão)")
        elif indicators['yield_curve_current'] < 0:
            score -= 1
            signals.append("⚠️ Curva de juros levemente invertida")
        elif indicators['yield_curve_current'] > 2:
            score += 1
            signals.append("✅ Curva de juros normal e íngreme")
    
    # Análise do VIX
    if 'vix_current' in indicators:
        if indicators['vix_current'] > 30:
            score -= 2
            signals.append("⚠️ VIX alto - medo no mercado")
        elif indicators['vix_current'] > 20:
            score -= 1
            signals.append("⚠️ VIX moderado - alguma tensão")
        else:
            score += 1
            signals.append("✅ VIX baixo - mercado tranquilo")
    
    # Análise do S&P 500
    if 'sp500_ytd_return' in indicators:
        if indicators['sp500_ytd_return'] < -15:
            score -= 2
            signals.append("⚠️ S&P 500 em bear market")
        elif indicators['sp500_ytd_return'] < -5:
            score -= 1
            signals.append("⚠️ S&P 500 com performance negativa")
        elif indicators['sp500_ytd_return'] > 15:
            score += 1
            signals.append("✅ S&P 500 com forte performance")
    
    # Análise de Volatilidade
    if 'sp500_volatility' in indicators:
        if indicators['sp500_volatility'] > 25:
            score -= 1
            signals.append("⚠️ Alta volatilidade no mercado")
        elif indicators['sp500_volatility'] < 15:
            score += 1
            signals.append("✅ Baixa volatilidade")
    
    # Determinação do ciclo
    if score >= 2:
        cycle = "🟢 EXPANSÃO"
        description = "Economia em crescimento, mercados otimistas"
        recommendations = [
            "Foco em growth stocks e tecnologia",
            "Reduzir posição em bonds de longo prazo",
            "Considerar REITs e commodities",
            "Aumentar exposição a small caps"
        ]
    elif score >= 0:
        cycle = "🟡 EXPANSÃO TARDIA"
        description = "Crescimento desacelerando, inflação possível"
        recommendations = [
            "Diversificar entre growth e value",
            "Considerar TIPS (inflation-protected securities)",
            "Manter posição defensiva em utilities",
            "Reduzir duration em bonds"
        ]
    elif score >= -2:
        cycle = "🟠 DESACELERAÇÃO"
        description = "Economia desacelerando, mercados cautelosos"
        recommendations = [
            "Migrar para value stocks e dividendos",
            "Aumentar posição em bonds de alta qualidade",
            "Considerar setores defensivos (utilities, consumer staples)",
            "Reduzir exposição a growth e tecnologia"
        ]
    else:
        cycle = "🔴 RECESSÃO"
        description = "Contração econômica, mercados em stress"
        recommendations = [
            "Posição defensiva: cash e treasuries",
            "Foco em dividend aristocrats",
            "Evitar high yield e junk bonds",
            "Considerar inverse ETFs para hedge"
        ]
    
    return {
        'cycle': cycle,
        'description': description,
        'score': score,
        'signals': signals,
        'recommendations': recommendations
    }

def display_cycle_analysis():
    """
    Executa a análise completa e exibe os resultados
    """
    print("🔍 ANALISANDO CICLO ECONÔMICO DO MERCADO AMERICANO...")
    print("=" * 60)
    
    # Coleta indicadores
    indicators = get_economic_cycle_indicators()
    
    # Exibe indicadores principais
    print("\n📊 INDICADORES ECONÔMICOS ATUAIS:")
    print("-" * 40)
    
    if 'yield_curve_current' in indicators and not np.isnan(indicators['yield_curve_current']):
        print(f"Curva de Juros (10Y-2Y): {indicators['yield_curve_current']:.2f}% ({indicators['yield_curve_trend']})")
    
    if 'vix_current' in indicators and not np.isnan(indicators['vix_current']):
        print(f"VIX (Índice do Medo): {indicators['vix_current']:.1f}")
    
    if 'sp500_ytd_return' in indicators and not np.isnan(indicators['sp500_ytd_return']):
        print(f"S&P 500 YTD: {indicators['sp500_ytd_return']:.1f}%")
    
    if 'sp500_volatility' in indicators and not np.isnan(indicators['sp500_volatility']):
        print(f"Volatilidade S&P 500: {indicators['sp500_volatility']:.1f}%")
    
    if 'dxy_current' in indicators and not np.isnan(indicators['dxy_current']):
        print(f"Índice do Dólar (DXY): {indicators['dxy_current']:.1f}")
    
    if 'oil_current' in indicators and not np.isnan(indicators['oil_current']):
        print(f"Petróleo WTI: ${indicators['oil_current']:.1f}")
    
    # Determina ciclo
    cycle_analysis = determine_economic_cycle(indicators)
    
    print(f"\n🎯 CICLO ECONÔMICO IDENTIFICADO:")
    print("-" * 40)
    print(f"Status: {cycle_analysis['cycle']}")
    print(f"Descrição: {cycle_analysis['description']}")
    print(f"Score de Confiança: {cycle_analysis['score']}")
    
    print(f"\n🔍 SINAIS IDENTIFICADOS:")
    print("-" * 40)
    for signal in cycle_analysis['signals']:
        print(f"  {signal}")
    
    print(f"\n💡 RECOMENDAÇÕES PARA CARTEIRA:")
    print("-" * 40)
    for i, rec in enumerate(cycle_analysis['recommendations'], 1):
        print(f"  {i}. {rec}")
    
    print("\n" + "=" * 60)
    print("✅ Análise concluída! Use essas informações para ajustar sua estratégia de alocação.")
    
    return cycle_analysis, indicators

# Executar análise
if __name__ == "__main__":
    cycle_result, raw_indicators = display_cycle_analysis()

🔍 ANALISANDO CICLO ECONÔMICO DO MERCADO AMERICANO...
Coletando dados dos Treasuries...
✅ Treasuries coletados: 10Y=4.20%, 2Y=4.18%
Erro ao coletar dados: "None of [DatetimeIndex(['2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11',\n               '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17',\n               '2023-08-18', '2023-08-21',\n               ...\n               '2025-07-21', '2025-07-22', '2025-07-23', '2025-07-24',\n               '2025-07-25', '2025-07-28', '2025-07-29', '2025-07-30',\n               '2025-07-31', '2025-08-01'],\n              dtype='datetime64[ns]', name='Date', length=498, freq=None)] are in the [columns]"

📊 INDICADORES ECONÔMICOS ATUAIS:
----------------------------------------
Curva de Juros (10Y-2Y): 0.02% (Normal)
VIX (Índice do Medo): 19.1
S&P 500 YTD: 38.1%
Volatilidade S&P 500: 16.1%
Índice do Dólar (DXY): 98.7
Petróleo WTI: $65.7

🎯 CICLO ECONÔMICO IDENTIFICADO:
----------------------------------------
Status: 🟢 EXPANSÃO
Descrição: E

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def get_cycle_based_assets(cycle_status="EXPANSÃO"):
    """
    Define ativos baseados no ciclo econômico identificado
    """
    
    # Dicionário com estratégias por ciclo
    strategies = {
        "EXPANSÃO": {
            "etfs": {
                # Tecnologia e Inovação
                "QQQ": {"name": "Invesco QQQ (NASDAQ-100)", "sector": "Technology", "risk": "High"},
                "XLK": {"name": "Technology Select SPDR", "sector": "Technology", "risk": "High"},
                "ARKK": {"name": "ARK Innovation ETF", "sector": "Innovation", "risk": "Very High"},
                "SOXX": {"name": "iShares Semiconductor ETF", "sector": "Semiconductors", "risk": "High"},
                
                # Crescimento e Small Caps
                "VTI": {"name": "Vanguard Total Stock Market", "sector": "Broad Market", "risk": "Medium"},
                "IWM": {"name": "iShares Russell 2000 (Small Cap)", "sector": "Small Cap", "risk": "High"},
                "VUG": {"name": "Vanguard Growth ETF", "sector": "Growth", "risk": "Medium-High"},
                
                # Setores Cíclicos
                "XLF": {"name": "Financial Select SPDR", "sector": "Financials", "risk": "Medium"},
                "XLI": {"name": "Industrial Select SPDR", "sector": "Industrials", "risk": "Medium"},
                "XLB": {"name": "Materials Select SPDR", "sector": "Materials", "risk": "Medium-High"},
                
                # REITs e Imóveis
                "VNQ": {"name": "Vanguard Real Estate ETF", "sector": "REITs", "risk": "Medium"},
                "XLRE": {"name": "Real Estate Select SPDR", "sector": "REITs", "risk": "Medium"},
                
                # Internacional
                "VXUS": {"name": "Vanguard Total International", "sector": "International", "risk": "Medium"},
                "VWO": {"name": "Vanguard Emerging Markets", "sector": "Emerging Markets", "risk": "High"}
            },
            
            "stocks": {
                # Magnificent 7 + AI Leaders
                "NVDA": {"name": "NVIDIA Corp", "sector": "AI/Semiconductors", "risk": "High"},
                "MSFT": {"name": "Microsoft Corp", "sector": "Cloud/AI", "risk": "Medium"},
                "GOOGL": {"name": "Alphabet Inc", "sector": "AI/Search", "risk": "Medium"},
                "AAPL": {"name": "Apple Inc", "sector": "Consumer Tech", "risk": "Medium"},
                "TSLA": {"name": "Tesla Inc", "sector": "EV/Energy", "risk": "Very High"},
                "META": {"name": "Meta Platforms", "sector": "Social/AI", "risk": "High"},
                "AMZN": {"name": "Amazon.com Inc", "sector": "E-commerce/Cloud", "risk": "Medium"},
                
                # Bancos (se beneficiam de juros)
                "JPM": {"name": "JPMorgan Chase", "sector": "Banking", "risk": "Medium"},
                "BAC": {"name": "Bank of America", "sector": "Banking", "risk": "Medium"},
                "WFC": {"name": "Wells Fargo", "sector": "Banking", "risk": "Medium"},
                
                # Growth e Inovação
                "CRM": {"name": "Salesforce Inc", "sector": "Cloud Software", "risk": "Medium"},
                "NFLX": {"name": "Netflix Inc", "sector": "Streaming", "risk": "Medium"},
                "AMD": {"name": "Advanced Micro Devices", "sector": "Semiconductors", "risk": "High"},
                
                # Small Cap Promissoras
                "PLTR": {"name": "Palantir Technologies", "sector": "AI/Data", "risk": "Very High"},
                "RBLX": {"name": "Roblox Corp", "sector": "Gaming/Metaverse", "risk": "Very High"}
            }
        },
        
        "EXPANSÃO TARDIA": {
            "etfs": {
                "VTI": {"name": "Vanguard Total Stock Market", "sector": "Broad Market", "risk": "Medium"},
                "VTV": {"name": "Vanguard Value ETF", "sector": "Value", "risk": "Medium"},
                "XLU": {"name": "Utilities Select SPDR", "sector": "Utilities", "risk": "Low"},
                "XLP": {"name": "Consumer Staples SPDR", "sector": "Consumer Staples", "risk": "Low"},
                "VTIP": {"name": "Vanguard Short-Term Inflation", "sector": "TIPS", "risk": "Low"}
            },
            "stocks": {
                "JNJ": {"name": "Johnson & Johnson", "sector": "Healthcare", "risk": "Low"},
                "PG": {"name": "Procter & Gamble", "sector": "Consumer Staples", "risk": "Low"},
                "KO": {"name": "Coca-Cola", "sector": "Consumer Staples", "risk": "Low"}
            }
        },
        
        "RECESSÃO": {
            "etfs": {
                "TLT": {"name": "iShares 20+ Year Treasury", "sector": "Long Treasuries", "risk": "Medium"},
                "SHY": {"name": "iShares 1-3 Year Treasury", "sector": "Short Treasuries", "risk": "Low"},
                "VYM": {"name": "Vanguard High Dividend Yield", "sector": "Dividend", "risk": "Medium"},
                "XLU": {"name": "Utilities Select SPDR", "sector": "Utilities", "risk": "Low"}
            },
            "stocks": {
                "WMT": {"name": "Walmart Inc", "sector": "Consumer Staples", "risk": "Low"},
                "KO": {"name": "Coca-Cola", "sector": "Consumer Staples", "risk": "Low"}
            }
        }
    }
    
    return strategies.get(cycle_status, strategies["EXPANSÃO"])

def analyze_asset_performance(symbols, period="6mo"):
    """
    Analisa performance e métricas dos ativos
    """
    results = []
    
    for symbol in symbols:
        try:
            # Download dados
            data = yf.download(symbol, period=period, progress=False)
            if data.empty:
                continue
                
            # Calcular métricas
            closes = data['Close'].dropna()
            returns = closes.pct_change().dropna()
            
            # Métricas básicas
            total_return = ((closes.iloc[-1] / closes.iloc[0]) - 1) * 100
            volatility = returns.std() * np.sqrt(252) * 100
            sharpe = (returns.mean() * 252) / (returns.std() * np.sqrt(252)) if returns.std() > 0 else 0
            
            # Máximo drawdown
            cumulative = (1 + returns).cumprod()
            rolling_max = cumulative.expanding().max()
            drawdown = ((cumulative - rolling_max) / rolling_max * 100).min()
            
            # RSI simples (últimos 14 dias)
            delta = closes.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs)).iloc[-1] if not rs.empty else 50
            
            # Volume médio (se disponível)
            avg_volume = data['Volume'].mean() if 'Volume' in data.columns else 0
            
            results.append({
                'symbol': symbol,
                'total_return': total_return,
                'volatility': volatility,
                'sharpe_ratio': sharpe,
                'max_drawdown': drawdown,
                'rsi': rsi,
                'avg_volume': avg_volume,
                'current_price': closes.iloc[-1]
            })
            
        except Exception as e:
            print(f"Erro ao analisar {symbol}: {e}")
            continue
    
    return pd.DataFrame(results)

def score_assets(df, cycle="EXPANSÃO"):
    """
    Pontua ativos baseado no ciclo econômico
    """
    if df.empty:
        return df
    
    df = df.copy()
    df['score'] = 0
    
    if cycle == "EXPANSÃO":
        # Em expansão, premiamos crescimento e momentum
        df['score'] += np.where(df['total_return'] > 20, 3, 
                       np.where(df['total_return'] > 10, 2,
                       np.where(df['total_return'] > 0, 1, -1)))  # Performance
        
        df['score'] += np.where(df['sharpe_ratio'] > 1.5, 2,
                       np.where(df['sharpe_ratio'] > 1, 1, 0))    # Risk-adjusted return
        
        df['score'] += np.where(df['rsi'] > 70, -1,
                       np.where(df['rsi'] < 30, 1, 0))           # RSI (avoid overbought)
        
        df['score'] += np.where(df['volatility'] < 30, 1, 0)     # Reasonable volatility
        
    elif cycle == "EXPANSÃO TARDIA":
        # Mais conservador, foco em qualidade
        df['score'] += np.where(df['total_return'] > 0, 2, -1)   # Performance positiva
        df['score'] += np.where(df['volatility'] < 20, 2, -1)    # Baixa volatilidade
        df['score'] += np.where(df['sharpe_ratio'] > 1, 2, 0)    # Boa relação risco/retorno
        df['score'] += np.where(df['max_drawdown'] > -15, 1, -1) # Menor drawdown
        
    else:  # RECESSÃO
        # Foco em preservação de capital
        df['score'] += np.where(df['volatility'] < 15, 3, -1)    # Muito baixa volatilidade
        df['score'] += np.where(df['max_drawdown'] > -10, 2, -1) # Proteção de capital
        df['score'] += np.where(df['total_return'] > -5, 1, -2)  # Evitar perdas grandes
    
    return df.sort_values('score', ascending=False)

def build_portfolio(cycle_status="EXPANSÃO", max_etfs=8, max_stocks=10):
    """
    Constrói portfólio automaticamente baseado no ciclo
    """
    print(f"🎯 CONSTRUINDO PORTFÓLIO PARA CICLO: {cycle_status}")
    print("=" * 60)
    
    # Obter ativos recomendados para o ciclo
    assets = get_cycle_based_assets(cycle_status)
    
    # Analisar ETFs
    print("📊 Analisando ETFs...")
    etf_symbols = list(assets['etfs'].keys())
    etf_analysis = analyze_asset_performance(etf_symbols)
    
    if not etf_analysis.empty:
        # Adicionar informações dos ETFs
        etf_analysis = etf_analysis.merge(
            pd.DataFrame.from_dict(assets['etfs'], orient='index').reset_index().rename(columns={'index': 'symbol'}),
            on='symbol', how='left'
        )
        
        # Pontuar e ranquear
        etf_scored = score_assets(etf_analysis, cycle_status)
        top_etfs = etf_scored.head(max_etfs)
        
        print(f"\n🏆 TOP {max_etfs} ETFs RECOMENDADOS:")
        print("-" * 50)
        for idx, row in top_etfs.iterrows():
            print(f"{row['symbol']:6} | {row['name'][:30]:30} | Score: {row['score']:2.0f} | Return: {row['total_return']:6.1f}% | Vol: {row['volatility']:5.1f}%")
    else:
        print("⚠️ Não foi possível analisar ETFs")
        top_etfs = pd.DataFrame()
    
    # Analisar Ações
    print(f"\n📈 Analisando Ações...")
    stock_symbols = list(assets['stocks'].keys())
    stock_analysis = analyze_asset_performance(stock_symbols)
    
    if not stock_analysis.empty:
        # Adicionar informações das ações
        stock_analysis = stock_analysis.merge(
            pd.DataFrame.from_dict(assets['stocks'], orient='index').reset_index().rename(columns={'index': 'symbol'}),
            on='symbol', how='left'
        )
        
        # Pontuar e ranquear
        stock_scored = score_assets(stock_analysis, cycle_status)
        top_stocks = stock_scored.head(max_stocks)
        
        print(f"\n🎯 TOP {max_stocks} AÇÕES RECOMENDADAS:")
        print("-" * 50)
        for idx, row in top_stocks.iterrows():
            print(f"{row['symbol']:6} | {row['name'][:30]:30} | Score: {row['score']:2.0f} | Return: {row['total_return']:6.1f}% | Vol: {row['volatility']:5.1f}%")
    else:
        print("⚠️ Não foi possível analisar ações")
        top_stocks = pd.DataFrame()
    
    # Sugestão de alocação
    print(f"\n💰 SUGESTÃO DE ALOCAÇÃO PARA {cycle_status}:")
    print("-" * 50)
    
    if cycle_status == "EXPANSÃO":
        print("🚀 Estratégia AGRESSIVA:")
        print("  • 60% ETFs (40% Tech/Growth + 20% Broad Market)")
        print("  • 35% Ações (25% Large Cap + 10% Small Cap)")
        print("  • 5% Cash para oportunidades")
        
    elif cycle_status == "EXPANSÃO TARDIA":
        print("⚖️ Estratégia BALANCEADA:")
        print("  • 50% ETFs (30% Broad Market + 20% Value/Defensive)")
        print("  • 30% Ações (Blue Chips e Dividendos)")
        print("  • 20% Bonds/TIPS para proteção")
        
    else:  # RECESSÃO
        print("🛡️ Estratégia DEFENSIVA:")
        print("  • 40% Bonds (Treasuries e High Grade)")
        print("  • 30% ETFs Defensivos (Utilities, Staples)")
        print("  • 20% Dividend Aristocrats")
        print("  • 10% Cash")
    
    return {
        'top_etfs': top_etfs,
        'top_stocks': top_stocks,
        'cycle': cycle_status
    }

def export_portfolio_summary(portfolio_data):
    """
    Exporta resumo do portfólio para análise
    """
    summary = {
        'cycle': portfolio_data['cycle'],
        'etf_count': len(portfolio_data['top_etfs']),
        'stock_count': len(portfolio_data['top_stocks']),
        'etf_symbols': portfolio_data['top_etfs']['symbol'].tolist() if not portfolio_data['top_etfs'].empty else [],
        'stock_symbols': portfolio_data['top_stocks']['symbol'].tolist() if not portfolio_data['top_stocks'].empty else [],
        'avg_etf_return': portfolio_data['top_etfs']['total_return'].mean() if not portfolio_data['top_etfs'].empty else 0,
        'avg_stock_return': portfolio_data['top_stocks']['total_return'].mean() if not portfolio_data['top_stocks'].empty else 0
    }
    
    print(f"\n📋 RESUMO EXPORTÁVEL:")
    print("-" * 30)
    print(f"Ciclo: {summary['cycle']}")
    print(f"ETFs Selecionados: {summary['etf_count']} ({summary['avg_etf_return']:.1f}% retorno médio)")
    print(f"Ações Selecionadas: {summary['stock_count']} ({summary['avg_stock_return']:.1f}% retorno médio)")
    print(f"Símbolos ETFs: {', '.join(summary['etf_symbols'])}")
    print(f"Símbolos Ações: {', '.join(summary['stock_symbols'])}")
    
    return summary

# Executar construção do portfólio
if __name__ == "__main__":
    # Usar o ciclo detectado anteriormente (ou definir manualmente)
    current_cycle = "EXPANSÃO"  # Baseado na análise anterior
    
    # Construir portfólio
    portfolio = build_portfolio(current_cycle, max_etfs=6, max_stocks=8)
    
    # Exportar resumo
    summary = export_portfolio_summary(portfolio)
    
    print("\n" + "=" * 60)
    print("✅ Portfólio construído! Símbolos prontos para análise detalhada.")

🎯 CONSTRUINDO PORTFÓLIO PARA CICLO: EXPANSÃO
📊 Analisando ETFs...
Erro ao analisar QQQ: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar XLK: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar ARKK: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar SOXX: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar VTI: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar IWM: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar VUG: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar XLF: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all

In [5]:
#GPT

import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from databento import Historical, Dataset

# 1) Carregar variável de ambiente
load_dotenv()
api_key = os.getenv("DB_API_KEY")
if not api_key:
    raise ValueError("API key não encontrada no .env!")

# 2) Inicializar o cliente
client = Historical(api_key=api_key)

# 3) Defina os ETFs de setor (como proxy)
SECTOR_ETFS = {
    "Consumer Discretionary": "XLY",
    "Consumer Staples":       "XLP",
    "Energy":                 "XLE",
    "Financials":             "XLF",
    "Health Care":            "XLV",
    "Industrials":            "XLI",
    "Materials":              "XLB",
    "Technology":             "XLK",
    "Utilities":              "XLU"
}

# 4) Baixar dados históricos para cada ETF
def get_sector_data(etfs, start_date, end_date):
    df_all = pd.DataFrame()
    for sector, symbol in etfs.items():
        print(f"Baixando dados de {symbol} ({sector})...")
        try:
            data = client.timeseries.get_range(
                dataset=Dataset.ETF_NBBO,  # Pode ser Dataset.IEX se preferir
                symbols=symbol,
                schema="ohlcv-1m",
                start=start_date,
                end=end_date,
                limit=1000
            )

            df = data.to_df()
            df["symbol"] = symbol
            df["sector"] = sector
            df["date"] = pd.to_datetime(df["ts_event"], unit="s").dt.date
            df_day = df.groupby("date")[["open", "high", "low", "close", "volume"]].mean().reset_index()
            df_day["sector"] = sector
            df_all = pd.concat([df_all, df_day], ignore_index=True)

        except Exception as e:
            print(f"Erro ao baixar {symbol}: {e}")

    return df_all

# 5) Definir intervalo
end_date = datetime.utcnow().date()
start_date = end_date - timedelta(days=30)

# 6) Obter os dados
dados_setoriais = get_sector_data(SECTOR_ETFS, start_date.isoformat(), end_date.isoformat())

# 7) Exibir resultado
print(dados_setoriais.head())


TypeError: Historical.__init__() got an unexpected keyword argument 'api_key'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests
import yfinance as yf
import databento as db  # Certifique-se de ter instalado: pip install databento

# Configurações iniciais
plt.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Configuração das APIs (substitua com suas chaves)
DATABENTO_API_KEY = "db-jpmKnEhu74j5tSBkDCma9WMuuu3sT "
FMP_API_KEY = "wzKZBImlIr1BLlc11ZXoh5DEkJdhkfnQ"

def get_databento_historical(ticker, start_date, end_date):
    """Obtém dados históricos da DataBento"""
    try:
        client = db.Historical(DATABENTO_API_KEY)
        
        # Converter símbolos para o formato DataBento
        db_symbol = {
            'SPY': 'SPY.AMEX',
            'TLT': 'TLT.NASDAQ',
            'GLD': 'GLD.NYSE',
            'IEF': 'IEF.NASDAQ',
            'QQQ': 'QQQ.NASDAQ',
            'DIA': 'DIA.NYSE',
            'SHY': 'SHY.NYSE'
        }.get(ticker, f"{ticker}.NYSE")  # Padrão para NYSE
        
        data = client.timeseries.get_range(
            dataset='XNAS.ITCH',  # Ajuste conforme necessário
            symbols=[db_symbol],
            schema='ohlcv-1d',
            start=start_date,
            end=end_date,
            stype_in='smart',
        )
        
        if not data.empty:
            df = data.reset_index()
            df['date'] = pd.to_datetime(df['ts_event'])
            df.set_index('date', inplace=True)
            return df['close']  # Usando preço de fechamento ajustado
        
        print(f"DataBento: Dados insuficientes para {ticker}")
        return None
    
    except Exception as e:
        print(f"Erro na DataBento para {ticker}: {str(e)}")
        return None

def get_fmp_historical(ticker, start_date, end_date):
    """Fallback para FMP"""
    try:
        url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?" \
              f"from={start_date}&to={end_date}&apikey={FMP_API_KEY}"
        
        response = requests.get(url)
        data = response.json()
        
        if 'historical' in data and len(data['historical']) > 0:
            df = pd.DataFrame(data['historical'])
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
            return df['adjClose']
        
        print(f"FMP: Dados insuficientes para {ticker}")
        return None
    
    except Exception as e:
        print(f"Erro na FMP para {ticker}: {str(e)}")
        return None

def get_yfinance_historical(ticker, start_date, end_date):
    """Último fallback para Yahoo Finance"""
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if not data.empty:
            return data['Adj Close'] if 'Adj Close' in data.columns else data['Close']
        return None
    except Exception as e:
        print(f"Erro no Yahoo Finance para {ticker}: {str(e)}")
        return None

def get_market_data(tickers, start_date, end_date):
    """Obtém dados hierarquicamente: DataBento > FMP > Yahoo Finance"""
    market_data = {}
    
    for ticker in tickers:
        print(f"\nObtendo {ticker}:")
        
        # 1. Tentar DataBento primeiro
        print("  Tentando DataBento...", end=' ')
        data = get_databento_historical(ticker, start_date, end_date)
        source = "DataBento" if data is not None else None
        
        # 2. Fallback para FMP
        if data is None:
            print("\n  Tentando FMP...", end=' ')
            data = get_fmp_historical(ticker, start_date, end_date)
            source = "FMP" if data is not None else None
        
        # 3. Último fallback para Yahoo Finance
        if data is None:
            print("\n  Tentando Yahoo Finance...", end=' ')
            data = get_yfinance_historical(ticker, start_date, end_date)
            source = "YFinance" if data is not None else None
        
        if data is not None:
            # Verificação de qualidade dos dados
            if isinstance(data.index, pd.DatetimeIndex) and len(data) > 10:
                market_data[ticker] = data
                print(f"✅ (Fonte: {source}, {len(data)} períodos)")
            else:
                print("❌ Dados insuficientes ou formato inválido")
        else:
            print("❌ Todas as fontes falharam")
    
    # Combinação final dos dados
    if market_data:
        # Encontrar o índice de data mais completo
        all_dates = pd.concat([pd.Series(idx) for idx in market_data.values()], axis=0).unique()
        reference_index = pd.to_datetime(sorted(all_dates))
        
        # Criar DataFrame alinhado
        aligned_data = pd.DataFrame(index=reference_index)
        for ticker, series in market_data.items():
            aligned_data[ticker] = series.reindex(reference_index).ffill().bfill()
        
        return aligned_data.dropna(how='all')
    return None

def analyze_market_cycle():
    """Analisa o ciclo atual do mercado com dados profissionais"""
    assets = {
        'SPY': 'S&P 500',
        'TLT': 'Títulos Longos',
        'GLD': 'Ouro',
        'IEF': 'Títulos Médios',
        'QQQ': 'Nasdaq',
        'DIA': 'Dow Jones',
        'SHY': 'Títulos Curtos'
    }
    
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')
    
    print("\n🔍 Iniciando análise profissional de ciclo de mercado")
    print(f"Período analisado: {start_date} a {end_date}")
    
    market_data = get_market_data(assets.keys(), start_date, end_date)
    
    if market_data is None or market_data.empty:
        print("\n⚠️ Erro crítico: Não foi possível obter dados suficientes")
        return None
    
    # Verificação final
    print("\n✅ Dados obtidos com sucesso:")
    print(f"Ativos disponíveis: {list(market_data.columns)}")
    print(f"Total de períodos: {len(market_data)}")
    print(f"Primeira data: {market_data.index[0].date()}")
    print(f"Última data: {market_data.index[-1].date()}")
    
    # Análise de desempenho relativo
    normalized = (market_data / market_data.iloc[0] * 100)
    
    # Plotagem profissional
    plt.figure(figsize=(14, 7))
    for col in normalized.columns:
        plt.plot(normalized.index, normalized[col], label=f"{col} ({assets[col]})", linewidth=2)
    
    plt.title('Desempenho Relativo Normalizado (Base 100)', fontsize=14)
    plt.ylabel('Retorno Normalizado', fontsize=12)
    plt.xlabel('Data', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    # Análise de momentum (6 meses)
    momentum = market_data.pct_change(126).iloc[-1].sort_values(ascending=False)
    
    # DataFrame formatado
    momentum_df = pd.DataFrame({
        'Ativo': momentum.index,
        'Retorno 6M': momentum.values,
        'Descrição': [assets[t] for t in momentum.index]
    })
    momentum_df['Retorno 6M'] = momentum_df['Retorno 6M'].apply(lambda x: f"{x:.1%}")
    
    print("\n📈 Análise de Momentum (6 meses):")
    print(momentum_df.to_markdown(index=False, tablefmt="grid"))
    
    # Diagnóstico avançado do ciclo
    if len(momentum) > 0:
        top_asset = momentum.index[0]
        top_return = momentum.iloc[0]
        
        if top_return > 0.15:
            if top_asset in ['SPY', 'QQQ', 'DIA']:
                cycle = "🟢 MERCADO EM ALTA (RISK-ON)"
                suggestion = """Estratégia recomendada:
                - Aumentar alocação em ações growth
                - Considerar alavancagem moderada
                - Reduzir duration de títulos"""
            elif top_asset in ['TLT', 'IEF', 'SHY']:
                cycle = "🟡 MERCADO DEFENSIVO"
                suggestion = """Estratégia recomendada:
                - Aumentar qualidade de crédito
                - Foco em ações defensivas (utilities, consumer staples)
                - Aumentar duration"""
            elif top_asset == 'GLD':
                cycle = "🔴 AMBIENTE INFLACIONÁRIO"
                suggestion = """Estratégia recomendada:
                - Aumentar exposição a commodities
                - Considerar TIPS e ativos reais
                - Reduzir duration nominal"""
        else:
            cycle = "🔵 MERCADO EM CORREÇÃO"
            suggestion = """Estratégia recomendada:
            - Aumentar liquidez
            - Considerar estratégias de hedge
            - Manter disciplina de stop-loss"""
        
        print(f"\n🔍 DIAGNÓSTICO DO CICLO: {cycle}")
        print(suggestion)
    
    return market_data

# Execução principal
if __name__ == "__main__":
    print("Iniciando análise profissional de mercado...")
    final_data = analyze_market_cycle()
    
    if final_data is not None:
        print("\n💼 Análise concluída com dados profissionais")
        print("Dados disponíveis na variável 'final_data'")
        print("\nPróximos passos sugeridos:")
        print("- Analisar correlações entre ativos")
        print("- Otimizar alocações por ciclo")
        print("- Implementar regras de rebalanceamento")
    else:
        print("\n❌ A análise não pôde ser concluída")

Iniciando análise profissional de mercado...

🔍 Iniciando análise profissional de ciclo de mercado
Período analisado: 2020-08-05 a 2025-08-04

Obtendo SPY:
  Tentando DataBento... Erro na DataBento para SPY: 400 auth_invalid_username_in_basic_auth
Invalid username in Basic auth ('db-jpmKnEhu74j5tSBkDCma9WMuuu3sT ').
documentation: https://databento.com/docs

  Tentando FMP... ✅ (Fonte: FMP, 1255 períodos)

Obtendo TLT:
  Tentando DataBento... Erro na DataBento para TLT: 400 auth_invalid_username_in_basic_auth
Invalid username in Basic auth ('db-jpmKnEhu74j5tSBkDCma9WMuuu3sT ').
documentation: https://databento.com/docs

  Tentando FMP... ✅ (Fonte: FMP, 1255 períodos)

Obtendo GLD:
  Tentando DataBento... Erro na DataBento para GLD: 400 auth_invalid_username_in_basic_auth
Invalid username in Basic auth ('db-jpmKnEhu74j5tSBkDCma9WMuuu3sT ').
documentation: https://databento.com/docs

  Tentando FMP... ✅ (Fonte: FMP, 1255 períodos)

Obtendo IEF:
  Tentando DataBento... Erro na DataBento p